loading required libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 5000)
import json
from openai import OpenAI
from dotenv import load_dotenv
from src.llm import doctor_prompt_disease_restricted_output_parser
from src.utils import convert_string_to_list,convert_clinical_case_summary,filterDepartment,getDepartmentStatistics,convert_cases_to_json
load_dotenv()

importing the dataset

In [ ]:
filePath="dataset\clinicallab\data_en.json"
with open(filePath, 'r', encoding='utf-8') as f:
            data = json.load(f)
print("number of total cases are",len(data))
print("each case have the following fields",list(data[0].keys()))

filtering the dataset

In [ ]:

keys_to_include = ["id",'clinical_department', 'principal_diagnosis', 'preliminary_diagnosis', 'diagnostic_basis', 'differential_diagnosis', 'treatment_plan', 'clinical_case_summary', 'imageological_examination', 'laboratory_examination', 'pathological_examination', 'therapeutic_principle']
df = pd.DataFrame([{key: d[key] for key in keys_to_include} for d in data])

In [ ]:
allDepartments=df['clinical_department'].value_counts()
print("number of departments available are",len(allDepartments))

print(allDepartments)

In [ ]:

df['preliminary_diagnosis'] = df['preliminary_diagnosis'].apply(convert_string_to_list)
df['diagnostic_basis'] = df['diagnostic_basis'].apply(convert_string_to_list)
df['differential_diagnosis'] = df['differential_diagnosis'].apply(convert_string_to_list)
df['treatment_plan'] = df['treatment_plan'].apply(convert_string_to_list)
df["clinical_case_summary"] = df["clinical_case_summary"].apply(convert_clinical_case_summary)

In [ ]:
department="neurosurgery department"
departmentdf=filterDepartment(df,department)
# departmentdf.to_json(f"{department}.json", orient='records', lines=False, indent=4)

In [ ]:
getDepartmentStatistics(departmentdf)

In [ ]:
uniqueDiseases=departmentdf["principal_diagnosis"].unique().tolist()
print(uniqueDiseases)
doctor="cardiologist"
model="gpt-4"

In [ ]:
caseNumber=40#68
row= departmentdf.iloc[caseNumber]
caseNumber=row.id
medicalHistory=row.clinical_case_summary
principalDiagnosis=row.principal_diagnosis
differentialDiagnosis=row.differential_diagnosis
print("the case id is",caseNumber)
print("\nthe principal diagnosis is",principalDiagnosis)
print("\ndifferential diagnosis is",differentialDiagnosis)
print("\nthe case history is ",medicalHistory)

In [ ]:
response=doctor_prompt_disease_restricted_output_parser(medicalHistory, model, uniqueDiseases)